In [33]:
import re
import sqlite3
import numpy as np
import pandas as pd
import tensorflow as tf

from time import time
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.layers import Embedding, LSTM, Dense, Dropout, Activation
from keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping, LambdaCallback
from keras.models import Sequential
import keras.utils as ku 
import numpy as np 


""" Custom Libs """
import Cleaner as c
#from embedding import ngram, generate_text_ngram

In [30]:
#Hyperparameters

epochs      = 300
dropout     = 0.2
l2_reg      = 3e-4
batch_sz    = 64
learn_rate  = 1e-3
beta_1      = 0.9
beta_2      = 0.999
epsilon     = None
decay_rate  = 0
amsgrad     = False
run_model   = True
seed = 0 #For reproducibility

np.random.seed(seed)

In [10]:
#Import Data
sqlite_file = '../../data/database/deeplearning.sqlite'
table_name  = 'tweets'
cd          = c.CleanData(sqlite_file, table_name)
q           ='SELECT * FROM {};'.format(table_name)

cd.set_table(q)
raw_data = cd.get_clean_table()
raw_data = raw_data.CleanText.values
data = ''
for x in raw_data:
    data += x + "\n"
    
sentences = [x.split(' ') for x in raw_data]

In [11]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(sentences, test_size = 0.2)
comments = train + test 

In [12]:
#Word2vec
from gensim.models import Word2Vec
window = 50 #no of context
min_count = 1 #min number of times word has to appear to be included
sg = 0 #skip gram = 1, cbow = 0
iteration = 100 #epochs
word_model = Word2Vec(comments, window = window, min_count = min_count, sg = sg, iter = iteration) 
pretrained_weights = word_model.wv.vectors
vocab_size, embedding_size = pretrained_weights.shape

In [13]:
print("Number of word vectors: {}".format(len(word_model.wv.vocab)))
word_model.wv.vocab

Number of word vectors: 11690


{'top': <gensim.models.keyedvectors.Vocab at 0x7fbf330ca588>,
 'ten': <gensim.models.keyedvectors.Vocab at 0x7fbf336d4c18>,
 'most': <gensim.models.keyedvectors.Vocab at 0x7fbf336d4b70>,
 'attractive': <gensim.models.keyedvectors.Vocab at 0x7fbf336d4b38>,
 'cities': <gensim.models.keyedvectors.Vocab at 0x7fbf336d4c50>,
 'for': <gensim.models.keyedvectors.Vocab at 0x7fbf336d4c88>,
 'the': <gensim.models.keyedvectors.Vocab at 0x7fbf336d4cc0>,
 'global': <gensim.models.keyedvectors.Vocab at 0x7fbf336d4cf8>,
 'workforce': <gensim.models.keyedvectors.Vocab at 0x7fbf336d4d30>,
 '1': <gensim.models.keyedvectors.Vocab at 0x7fbf336d4d68>,
 '7': <gensim.models.keyedvectors.Vocab at 0x7fbf336d4da0>,
 'via': <gensim.models.keyedvectors.Vocab at 0x7fbf336d4dd8>,
 'a': <gensim.models.keyedvectors.Vocab at 0x7fbf336d4e10>,
 'ship': <gensim.models.keyedvectors.Vocab at 0x7fbf336d4e48>,
 'without': <gensim.models.keyedvectors.Vocab at 0x7fbf336d4e80>,
 'informal': <gensim.models.keyedvectors.Vocab at 0

In [14]:
word_model.wv.most_similar_cosmul(positive=['machine', 'learning'], ) #Test

[('deep', 0.4307817816734314),
 ('techniques', 0.420337051153183),
 ('recent', 0.40253016352653503),
 ('adaptable', 0.3911099433898926),
 ('sequential', 0.39025354385375977),
 ('ion', 0.3828776180744171),
 ('touting', 0.3813908100128174),
 ('stoica', 0.38011375069618225),
 ('humancentered', 0.3774866759777069),
 ('horizon', 0.3762533962726593)]

In [15]:
def word2idx(word):
  return word_model.wv.vocab[word].index
def idx2word(idx):
  return word_model.wv.index2word[idx]

In [16]:
print('\nPreparing the data for LSTM...')
max_sentence_len = max([len(sentence) for sentence in sentences])

train_x = np.zeros([len(sentences), max_sentence_len], dtype=np.int32)
train_y = np.zeros([len(sentences)], dtype=np.int32)
for i, sentence in enumerate(sentences):
  for t, word in enumerate(sentence[:-1]):
    train_x[i, t] = word2idx(word)
  train_y[i] = word2idx(sentence[-1])
print('train_x shape:', train_x.shape)
print('train_y shape:', train_y.shape)


Preparing the data for LSTM...
train_x shape: (7501, 49)
train_y shape: (7501,)


In [17]:
print('\nTraining LSTM...')
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_size, weights=[pretrained_weights]))
model.add(LSTM(200, return_sequences = True))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dense(units=vocab_size))
model.add(Activation('softmax'))
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')


Training LSTM...


In [34]:
def create_model(weights):
    model = Sequential()
    model.add(Embedding(input_dim=weights.shape[0], output_dim=weights.shape[1], weights=[weights]))
    model.add(LSTM(512, return_sequences = True))
    if dropout != 0:
        model.add(Dropout(dropout))
        model.add(LSTM(256))
    else:
        model.add(LSTM(256))
    if l2_reg != 0:
        model.add(Dense(weights.shape[0], activation = 'softmax', bias_regularizer = l2(l2_reg)))
    else:
        model.add(Dense(weights.shape[0], activation = 'softmax'))
    model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics=['accuracy'])
    checkpointer = ModelCheckpoint(filepath='model'
                                   + '/single-user-model-{}'.format(time()) + '-{epoch:02d}.hdf5', verbose = 1)
    tensorboard = TensorBoard(log_dir = 'tb-logs/{}'.format(time()))
    earlystop = EarlyStopping(monitor='loss', min_delta=0, patience=100, verbose=0, mode='min')
    return(model, checkpointer, tensorboard, earlystop)

In [18]:
def sample(preds, temperature=1.0):
  if temperature <= 0:
    return np.argmax(preds)
  preds = np.asarray(preds).astype('float64')
  preds = np.log(preds) / temperature
  exp_preds = np.exp(preds)
  preds = exp_preds / np.sum(exp_preds)
  probas = np.random.multinomial(1, preds, 1)
  return np.argmax(probas)

In [19]:
def generate_next(text, num_generated=10):
  word_idxs = [word2idx(word) for word in text.lower().split()]
  for i in range(num_generated):
    prediction = model.predict(x=np.array(word_idxs))
    idx = sample(prediction[-1], temperature=0.7)
    word_idxs.append(idx)
    if idx2word(idx) == '.':
        return ' '.join(idx2word(idx) for idx in word_idxs)
  return ' '.join(idx2word(idx) for idx in word_idxs)

In [20]:
def on_epoch_end(epoch, _):
  print('\nGenerating text after epoch: %d' % epoch)
  texts = [
    'data',
    'deep',
    'machine',
  ]
  for text in texts:
    sample = generate_next(text)
    print('%s... -> %s' % (text, sample))

In [37]:
model, checkpointer, tensorboard, earlystop = create_model(pretrained_weights)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, None, 100)         1169000   
_________________________________________________________________
lstm_10 (LSTM)               (None, None, 512)         1255424   
_________________________________________________________________
dropout_5 (Dropout)          (None, None, 512)         0         
_________________________________________________________________
lstm_11 (LSTM)               (None, 256)               787456    
_________________________________________________________________
dense_4 (Dense)              (None, 11690)             3004330   
Total params: 6,216,210
Trainable params: 6,216,210
Non-trainable params: 0
_________________________________________________________________


In [39]:
if run_model == True:
    if tf.test.is_gpu_available():
        model.fit(x = train_x, y = train_y, 
                  epochs = epochs, 
                  batch_size = batch_sz,
                  validation_split = 0.25,
                  verbose = 1, 
                  callbacks=[checkpointer, tensorboard, earlystop, LambdaCallback(on_epoch_end=on_epoch_end)])

Train on 5625 samples, validate on 1876 samples
Epoch 1/300
5625/5625 [==============================] - 13s 2ms/step - loss: 7.6792 - acc: 0.1246 - val_loss: 6.8544 - val_acc: 0.3129

Epoch 00001: saving model to model/single-user-model-1545209186.0644634-01.hdf5

Generating text after epoch: 0
data... -> data potentially failed 4pm teste bottle princeton team paradigmread network tech
deep... -> deep pare herd 2027 mail beijing cannabis openstack tlvision heed vault
machine... -> machine sacrifice nga establishment lasted rains 20milliards mindful canal 776 gathering
Epoch 2/300
5625/5625 [==============================] - 12s 2ms/step - loss: 6.5692 - acc: 0.1260 - val_loss: 7.1513 - val_acc: 0.3129

Epoch 00002: saving model to model/single-user-model-1545209186.0644634-02.hdf5

Generating text after epoch: 1
data... -> data heuristicsusing frontline done price spotted individually destroyed feynman hidden hottest
deep... -> deep 20th corrupted digitallyenabled variables macys shop

5625/5625 [==============================] - 14s 3ms/step - loss: 5.9804 - acc: 0.1319 - val_loss: 7.5199 - val_acc: 0.2468

Epoch 00016: saving model to model/single-user-model-1545209186.0644634-16.hdf5

Generating text after epoch: 15
data... -> data limitedread unclean nayak indices lord canadas tabletbereich was ann debts
deep... -> deep hays do1ygzdcg4 exemple paiement bold cryptography deny a2b momentum impede
machine... -> machine tumblr kick superstars economa transformational differentially kolhapur embedding 865 judgement
Epoch 17/300
5625/5625 [==============================] - 14s 2ms/step - loss: 5.8718 - acc: 0.1339 - val_loss: 7.6220 - val_acc: 0.2260

Epoch 00017: saving model to model/single-user-model-1545209186.0644634-17.hdf5

Generating text after epoch: 16
data... -> data declared depending krishna 12 spike geeksforgeeks lovers youin clothes churn
deep... -> deep contrast occur etail 144 mentorshorseface 1st lies amused segran city
machine... -> machine greatest 

5625/5625 [==============================] - 14s 3ms/step - loss: 3.5969 - acc: 0.3444 - val_loss: 8.4195 - val_acc: 0.1466

Epoch 00031: saving model to model/single-user-model-1545209186.0644634-31.hdf5

Generating text after epoch: 30
data... -> data rajan crossborder 1080p boosts via puanodolcs communist transforming how sustained
deep... -> deep readmission ym7zlui5mg costs anatomy broady standards transaction printing6 unifying phases
machine... -> machine highlyscalable 564 sondes preparring bottleneck 84 debunking mfa participating reasoning
Epoch 32/300
5625/5625 [==============================] - 14s 2ms/step - loss: 3.3898 - acc: 0.3717 - val_loss: 8.6142 - val_acc: 0.1343

Epoch 00032: saving model to model/single-user-model-1545209186.0644634-32.hdf5

Generating text after epoch: 31
data... -> data bwnpeekghb then pretty shawn goldieblox cfdjpf7a38 fruit behalf croise ikea
deep... -> deep yangqing raging aimed briefing on australian each fastreforming elite article
machine

5625/5625 [==============================] - 15s 3ms/step - loss: 1.0560 - acc: 0.8747 - val_loss: 9.3164 - val_acc: 0.1482

Epoch 00046: saving model to model/single-user-model-1545209186.0644634-46.hdf5

Generating text after epoch: 45
data... -> data hans hurry pants ii elite day nokia tg for currents
deep... -> deep lightstraffic ng 4irraglan specialep317 might 737mt synced focusing attorney dinfluence
machine... -> machine gamification intentions customers als rekorde century obkzv9u87y prone 91 excerpts
Epoch 47/300
5625/5625 [==============================] - 15s 3ms/step - loss: 0.9595 - acc: 0.8898 - val_loss: 9.2827 - val_acc: 0.1546

Epoch 00047: saving model to model/single-user-model-1545209186.0644634-47.hdf5

Generating text after epoch: 46
data... -> data transformative hastings testbot sydney appreciative campaign rooms haskew traded cc
deep... -> deep pandasis dashboards dybceoweeg im focussed 30th architectures subscribe dynamical vulnerabilities
machine... -> machin

5625/5625 [==============================] - 16s 3ms/step - loss: 0.1541 - acc: 0.9881 - val_loss: 9.7313 - val_acc: 0.1493

Epoch 00061: saving model to model/single-user-model-1545209186.0644634-61.hdf5

Generating text after epoch: 60
data... -> data dhandapani unfolding methodologies quantiply jeffrey coaching3 510 wisdom deadly 147
deep... -> deep uhhuh trademark systems transport unconscious using recognised digest removed z
machine... -> machine atms traffic avoided 1229 element sep tsa fourth rockets love
Epoch 62/300
5625/5625 [==============================] - 17s 3ms/step - loss: 0.1490 - acc: 0.9868 - val_loss: 9.7568 - val_acc: 0.1503

Epoch 00062: saving model to model/single-user-model-1545209186.0644634-62.hdf5

Generating text after epoch: 61
data... -> data cofoundercto distinguishes ups wallsby experiencedean faced hire sensed 32 insects
deep... -> deep avatarstyle outpacing marketeurs risen sweetie line franchit bigdl headlearning qualitative
machine... -> machine s

...

machine... -> machine comptes via options allows agile have prioritization cultures multisixfigure awards
Epoch 241/300
5625/5625 [==============================] - 16s 3ms/step - loss: 0.0148 - acc: 0.9938 - val_loss: 10.4696 - val_acc: 0.1551

Epoch 00241: saving model to model/single-user-model-1545209186.0644634-241.hdf5

Generating text after epoch: 240
data... -> data postgres krizhevsky valueprecision algebra relationshipbuilding exhibitor centric westeuropa flagship paloaltohighschool
deep... -> deep fvt2r99axm giftsuperbe butt practice thet crave focuses operate voted mellanox
machine... -> machine pigmymhsyt oneven yugyi1ifre amounts sheer dauern factoring solar thoughts wireless
Epoch 242/300
5625/5625 [==============================] - 16s 3ms/step - loss: 0.0145 - acc: 0.9938 - val_loss: 10.4198 - val_acc: 0.1583

Epoch 00242: saving model to model/single-user-model-1545209186.0644634-242.hdf5

Generating text after epoch: 241
data... -> data wdlwnleald headstart mancheste

5625/5625 [==============================] - 16s 3ms/step - loss: 0.0812 - acc: 0.9819 - val_loss: 10.1499 - val_acc: 0.1610

Epoch 00256: saving model to model/single-user-model-1545209186.0644634-256.hdf5

Generating text after epoch: 255
data... -> data perceived dataset indicator mentoring sap travelling unverhofft earl habla readmission
deep... -> deep ellsworth newtakanawa shoppingby wide outsource territory outer palmerston libreservice poisoning
machine... -> machine meetup computation animals zphfmmbrf8 workers hiranandani wave walks peter precaution
Epoch 257/300
5625/5625 [==============================] - 17s 3ms/step - loss: 0.0378 - acc: 0.9906 - val_loss: 10.2594 - val_acc: 0.1471

Epoch 00257: saving model to model/single-user-model-1545209186.0644634-257.hdf5

Generating text after epoch: 256
data... -> data journal forerunners via parkour director amazing teched tour matter club
deep... -> deep dynamically reliance drew selection pass popularity mustfollow anna specul

5625/5625 [==============================] - 16s 3ms/step - loss: 0.0149 - acc: 0.9941 - val_loss: 10.2988 - val_acc: 0.1572

Epoch 00271: saving model to model/single-user-model-1545209186.0644634-271.hdf5

Generating text after epoch: 270
data... -> data walked moscone tsapre prestigious arms open delegate 16 wonder anyway
deep... -> deep mr apart 3714 accessible enginepeter 25bn lebron nave july steering
machine... -> machine customeroriented ranking aboutthe paralysis 522 nederland hatred retraining destiny impbut
Epoch 272/300
5625/5625 [==============================] - 15s 3ms/step - loss: 0.0146 - acc: 0.9941 - val_loss: 10.2757 - val_acc: 0.1599

Epoch 00272: saving model to model/single-user-model-1545209186.0644634-272.hdf5

Generating text after epoch: 271
data... -> data embb continuing specialist 2023by stats bm9zlmral5 chasing concrete wuzhen vr
deep... -> deep route terry contribute edition friends privacy seriesgreat humanoid dubai compaera
machine... -> machine hot de

machine... -> machine tale kvfedokzll revolutionvia subjective conducting weight explosion em topicsdigital facial
Epoch 286/300
5625/5625 [==============================] - 16s 3ms/step - loss: 0.0142 - acc: 0.9941 - val_loss: 10.4105 - val_acc: 0.1557

Epoch 00286: saving model to model/single-user-model-1545209186.0644634-286.hdf5

Generating text after epoch: 285
data... -> data area feature agnostic morgan announced structure herrell experience youin image
deep... -> deep examine spectrum yearso verticals tutorials page klut5esg7i 20th nast 42
machine... -> machine perceptions debunked pst start cloudcomputing item irrelevant software stan overly
Epoch 287/300
5625/5625 [==============================] - 17s 3ms/step - loss: 0.0143 - acc: 0.9934 - val_loss: 10.4056 - val_acc: 0.1535

Epoch 00287: saving model to model/single-user-model-1545209186.0644634-287.hdf5

Generating text after epoch: 286
data... -> data neue youtube for uberrich digitallynetworked full languages christie 

machine... -> machine muditsaxena 186 communicating successful 15th atempo dos iot adweek healthcare


In [40]:

word_model = Word2Vec.load("word2vec.model")

In [43]:
word_model.wv.vocab['just']